In [46]:
import tushare as ts
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score
import datetime

today = int(datetime.datetime.now().strftime("%Y%m%d"))


In [47]:
today

20230418

In [48]:
# 将token替换为自己的tushare token
ts.set_token('7a744812e15bc5fd281125c887d61c224bcaced5be1b520cb22a1a51')
pro = ts.pro_api()

In [68]:
# 拉取数据
df = pro.fut_daily(**{
    "trade_date": "",
    "ts_code": "AG2306.SHF",
    "exchange": "SHFE",
    "start_date": today-10000,
    "end_date": today,
    "limit": 10000,
    "offset": ""
}, fields=[
    "ts_code",
    "trade_date",
    "pre_close",
    "pre_settle",
    "open",
    "high",
    "low",
    "close",
    "settle",
    "change1",
    "change2",
    "vol",
    "amount",
    "oi",
    "oi_chg",
    "delv_settle"
])

df.insert(0, 'number', range(1, len(df)+1))        

In [69]:
df.to_csv('silver_futures.csv', index=False)

In [70]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 读入数据
df = pd.read_csv('silver_futures.csv')

# 构建特征矩阵
features = ['change1', 'open', 'close', 'oi', 'vol']
X = np.array(df[features])
y = np.array(df['change1'] > 0)

# 划分训练集和测试集
train_size = int(0.7 * len(df))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# 建立随机森林模型
rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
rfc.fit(X_train, y_train)

# 预测测试集
y_pred = rfc.predict(X_test)

# 评估模型准确度
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 1.0


In [71]:
# 假设你已经得到了一个新的数据点，保存在一个名为new_data的DataFrame中
new_data = pd.DataFrame({'change1': [84], 
                         'open': [4777], 
                         'close': [4776], 
                         'oi': [53], 
                         'vol': [824]})

# 对新数据进行预测
y_pred = rfc.predict_proba(new_data)

# 输出涨跌概率
print('下跌概率:', y_pred[:, 0])
print('上涨概率:', y_pred[:, 1])


下跌概率: [0.10532748]
上涨概率: [0.89467252]


c:\Users\DuYih\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


In [72]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# 加载数据
data = pd.read_csv('silver_futures.csv')

# 构建特征矩阵
features = ['open', 'close', 'high', 'low']
X = []
for i in range(3, len(data)):
    x_i = data.loc[i-3:i-1, features].values.reshape(-1)
    X.append(x_i)
X = np.array(X)

# 标记目标变量
y = np.where(data['change1'][3:] > 0, 1, 0)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 训练模型
rfc = RandomForestClassifier(n_estimators=100, random_state=42)
rfc.fit(X_train, y_train)

# 评估模型
y_pred = rfc.predict(X_test)
print('准确率：', accuracy_score(y_test, y_pred))
print('精确率：', precision_score(y_test, y_pred))
print('召回率：', recall_score(y_test, y_pred))



准确率： 0.5853658536585366
精确率： 0.6086956521739131
召回率： 0.6363636363636364


In [73]:
# 构造新数据
new_data = data.loc[len(data)-3:len(data)-1, features].values.reshape(-1, 12)

# 预测涨跌概率
y_pred = rfc.predict_proba(new_data)

# 输出涨跌概率
print('下跌概率:', y_pred[0, 0])
print('上涨概率:', y_pred[0, 1])


下跌概率: 0.4
上涨概率: 0.6
